# Import Library

In [1]:
import pandas as pd, numpy as np, requests, os, json
from bs4 import BeautifulSoup as BS

# Static Data

In [2]:
source = 'https://www.bajajallianz.com/health-insurance-plans/health-insurance-documents.html'
author = 'Bajaj Allianz'

# Scraper

In [3]:
content =  requests.get('https://www.bajajallianz.com/health-insurance-plans/health-insurance-documents.html').content
soup = BS(content, 'lxml')

table_data = soup.find('section', {'class': 'typography-section'}).find('tbody').find_all('tr')


In [4]:
len(table_data)

25

In [5]:
data = {}
columns_names = {
    1:"Brochure", 2:"Customer Information Sheet", 3:"Policy Wordings", 4:"Proposal Form"
}

for row in table_data:
    cols = row.find_all('td') 
    data[cols[0].text.strip()] = {
        columns_names[idx]: [
            link['href']
            for link in col.find_all('a')
        ]
        for idx,col in enumerate(cols[1:],start=1)
    }


# Save data

In [6]:
os.makedirs('../data', exist_ok=True)

with open('../data/bajaj_insurance_documents_statewise.json', 'w') as f:
    json.dump(
        {
            "source": source,
            "author": author,
            "data": data
        }, 
        f, indent=4
    )